In [1]:
# BASIC SoftMax Classifier
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""A very simple MNIST classifier.

See extensive documentation at
http://tensorflow.org/tutorials/mnist/beginners/index.md
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import math
# pylint: disable=missing-docstring
import os.path
import sys
from six.moves import xrange  # pylint: disable=redefined-builtin
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# MNIST Globals
# The MNIST dataset has 10 classes, representing the digits 0 through 9.
gNUM_CLASSES = 10

# The MNIST images are always 28x28 pixels.
gIMAGE_SIZE = 28
gIMAGE_PIXELS = gIMAGE_SIZE * gIMAGE_SIZE

# Parameter Globals
gMnistDir = "/home/data/asarcar-data/tmp/MNIST"
# 'Initial learning rate.'
gInputDataDir           =  gMnistDir + "/data"
# 'Number of steps to run trainer.'
gLogDir             = gMnistDir + "/logs"
# 'Number of units in hidden layer 1.'
gLearningRate   = 0.01
# 'Number of units in hidden layer 2.'
gMaxSteps       = 2000
# 'Batch size.  Must divide evenly into the dataset sizes.'
gHidden1        = 128
# 'Directory to put the input data.'
gHidden2        = 32
# 'Directory to put the log data.'
gBatchSize      = 100
# 'If true, uses fake data for unit testing.',
gFakeData       = False
# 'Keep probability for training dropout.'
gDropOut = 0.9

In [0]:
# Import Data
mnist_data_dir = gInputDataDir
mnist_data = input_data.read_data_sets(mnist_data_dir, one_hot=True, fake_data=gFakeData)

In [4]:
#  Convolutional Neural Net Classifier
# Noise in W breaks symmetry. Otherwise: backpropagation does not
# function: Error(l) = W'(l+1)*errors(l+1)*grad(f(l)) = 0. 
def weight_variable(shape):
  initial = tf.truncated_normal(shape,stddev=0.1)
  return tf.Variable(initial)

In [5]:
# ReLU require +ve initial bias to avoid "dead" neurons.
def bias_variable(shape):
  initial = tf.constant(0.1,shape=shape)
  return tf.Variable(initial)

In [6]:
# Input placeholders
with tf.name_scope('input'):
  # Create the model
  x = tf.placeholder(tf.float32, [None, 784])
  # Define loss and optimizer
  y_ = tf.placeholder(tf.float32, [None, 10])

In [177]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.matmul(x, W) + b

In [178]:
# The raw formulation of cross-entropy,
#
#   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
#                                 reduction_indices=[1]))
#
# can be numerically unstable.
#
# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [179]:
init = tf.global_variables_initializer()

In [180]:
sess = tf.InteractiveSession()
sess.run(init)
# Train
for _ in range(1000):
  batch_xs, batch_ys = mnist_data.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [181]:
# Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist_data.test.images,
				    y_: mnist_data.test.labels}))

0.9171


In [0]:
# convolutions uses a stride of one and are zero padded
# output is the same size as the input. 
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [147]:
# plain old max pooling over 2x2 blocks in 2nd & 3rd Dimension
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [148]:
# Reshape image to a 4D Tensor (D1,D2,D3,D4) == (N,28,28,1)
# where N is number of images submitted in the batch
# -1 used to infer the shape to map to same tensor size
x_image = tf.reshape(x, [-1,28,28,1])

In [149]:
# First Convolution Layer: (5,5,1) => 32 features
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# Convole image(W,b) => ReLU => MaxPool
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
# Image Pooled: (N,28,28,32) => (N,14,14,32)
h_pool1 = max_pool_2x2(h_conv1)

In [150]:
# Second Convolution Layer: (5,5,32) => 64 features
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

# Convole image(W,b) => ReLU => MaxPool
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
# Image Pooled: (N,14,14,64) => (N,7,7,64)
h_pool2 = max_pool_2x2(h_conv2)

In [151]:
# Fully Connected Layer: (7,7,64) => 1024 features
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

# Reshape to N vector(s) => (W,b) => ReLU
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [152]:
# Apply Dropout
# placeholder: allows turning on during training and off during testing.
# tf.nn.dropout: automatically handles scaling neuron outputs
# in addition to masking. so dropout works without any additional scaling.
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [153]:
# Final Output Layer: (N,1024) => (W,b) => (N,10)
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [154]:
# The raw formulation of cross-entropy,
#
#   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
#                                 reduction_indices=[1]))
#
# can be numerically unstable.
#
# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
# Use ADAM Optimizer (step size smartly taken)
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [155]:
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [156]:
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)

In [157]:
log_iter = 2000
max_iter = 20000
# Train
for i in range(max_iter):
  batch = mnist_data.train.next_batch(50)
  # sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5}) 
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

  # Feed: Dropout probability when feeding Tensor Computation Graph
  # Log every 100th iteration during training
  if i%log_iter == 0:
    train_accuracy = accuracy.eval(feed_dict={
      x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))


step 18000, training accuracy 1


step 16000, training accuracy 1


step 14000, training accuracy 1


step 12000, training accuracy 1


step 10000, training accuracy 1


step 8000, training accuracy 1


step 6000, training accuracy 1


step 4000, training accuracy 1


step 2000, training accuracy 1


step 0, training accuracy 0.18


In [158]:
print("test accuracy %g"%accuracy.eval(feed_dict={
  x: mnist_data.test.images, y_: mnist_data.test.labels, keep_prob: 1.0}))

test accuracy 0.9926


In [159]:
#  2 Stage Neural Net + Softmax Classifier

# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Builds the MNIST network.
Implements the inference/loss/training pattern for model building.
1. inference() - Builds the model as far as is required for running the network
forward to make predictions.
2. loss() - Adds to the inference model the layers required to generate loss.
3. training() - Adds to the loss model the Ops required to generate and
apply gradients.

This file is used by the various "fully_connected_*.py" files and not meant to
be run.
"""

'Builds the MNIST network.\nImplements the inference/loss/training pattern for model building.\n1. inference() - Builds the model as far as is required for running the network\nforward to make predictions.\n2. loss() - Adds to the inference model the layers required to generate loss.\n3. training() - Adds to the loss model the Ops required to generate and\napply gradients.\n\nThis file is used by the various "fully_connected_*.py" files and not meant to\nbe run.\n'

In [170]:
mnist_data = input_data.read_data_sets(mnist_data_dir, gFakeData)

Extracting /home/data/asarcar-data/tmp/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /home/data/asarcar-data/tmp/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /home/data/asarcar-data/tmp/MNIST_data/t10k-labels-idx1-ubyte.gz


Extracting /home/data/asarcar-data/tmp/MNIST_data/train-images-idx3-ubyte.gz


In [160]:
def inference(images, hidden1_units, hidden2_units):
  """Build the MNIST model up to where it may be used for inference.
  Args:
    images: Images placeholder, from inputs().
    hidden1_units: Size of the first hidden layer.
    hidden2_units: Size of the second hidden layer.

  Returns:
    softmax_linear: Output tensor with the computed logits.
  """
  # Hidden 1
  with tf.name_scope('hidden1'):
    wt_init = tf.truncated_normal([gIMAGE_PIXELS, hidden1_units],
				  stddev=1.0 / math.sqrt(float(gIMAGE_PIXELS)))
    weights = tf.Variable(wt_init, name='weights')
    bi_init = tf.zeros([hidden1_units])
    biases = tf.Variable(bi_init, name='biases')
    hidden1 = tf.nn.relu(tf.matmul(images, weights) + biases)
  # Hidden 2
  with tf.name_scope('hidden2'):
    wt_init = tf.truncated_normal([hidden1_units, hidden2_units],
				  stddev=1.0 / math.sqrt(float(hidden1_units)))
    weights = tf.Variable(wt_init, name='weights')
    bi_init = tf.zeros([hidden2_units])
    biases = tf.Variable(bi_init, name='biases')
    hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
  # Linear
  with tf.name_scope('softmax_linear'):
    wt_init = tf.truncated_normal([hidden2_units, gNUM_CLASSES],
				  stddev=1.0 / math.sqrt(float(hidden2_units)))
    weights = tf.Variable(wt_init, name='weights')
    bi_init = tf.zeros([gNUM_CLASSES]),
    biases = tf.Variable(bi_init, name='biases')
    logits_op = tf.matmul(hidden2, weights) + biases
  return logits_op

In [161]:
def loss(logits, labels):
  """Calculates the loss from the logits and the labels.
  Args:
    logits: Logits tensor, float - [batch_size, gNUM_CLASSES].
    labels: Labels tensor, int32 - [batch_size].

  Returns:
    loss: Loss tensor of type float.
  """
  labels = tf.to_int64(labels)
  cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
      logits, labels, name='xentropy')
  loss_op = tf.reduce_mean(cross_entropy, name='xentropy_mean')
  return loss_op

In [162]:
def training(loss_op, learning_rate):
  """Sets up the training Ops.
  Creates a summarizer to track the loss over time in TensorBoard.

  Creates an optimizer and applies the gradients to all trainable variables.

  The Op returned by this function is what must be passed to the
  `sess.run()` call to cause the model to train.

  Args:
    loss: Loss tensor, from loss().
    learning_rate: The learning rate to use for gradient descent.

  Returns:
    train_op: The Op for training.
  """
  # Add a scalar summary for the snapshot loss.
  # tf.summary.scalar('loss', loss_op)
  tf.summary.scalar(loss_op.op.name, loss_op)
  # Create the gradient descent optimizer with the given learning rate.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  # Create a variable to track the global step.
  global_step = tf.Variable(0, name='global_step', trainable=False)
  # Use the optimizer to apply the gradients that minimize the loss
  # (and also increment the global step counter) as a single training step.
  train_op = optimizer.minimize(loss_op, global_step=global_step)
  return train_op

In [163]:
def evaluation(logits, labels):
  """Evaluate the quality of the logits at predicting the label.

  Args:
    logits: Logits tensor, float - [batch_size, gNUM_CLASSES].
    labels: Labels tensor, int32 - [batch_size], with values in the
      range [0, gNUM_CLASSES).

  Returns:
    A scalar int32 tensor with the number of examples (out of batch_size)
    that were predicted correctly.
  """
  # For a classifier model, we can use the in_top_k Op.
  # It returns a bool tensor with shape [batch_size] that is true for
  # the examples where the label is in the top k (here k=1)
  # of all logits for that example.
  correct = tf.nn.in_top_k(logits, labels, 1)
  # Return the number of true entries.
  return tf.reduce_sum(tf.cast(correct, tf.int32))

In [164]:
def placeholder_inputs(batch_size):
  """Generate placeholder variables to represent the input tensors.
  These placeholders are used as inputs by the rest of the model building
  code and will be fed from the downloaded data in the .run() loop, below.
  Args:
    batch_size: The batch size will be baked into both placeholders.

  Returns:
    images_placeholder: Images placeholder.
    labels_placeholder: Labels placeholder.
  """
  # Note that the shapes of the placeholders match the shapes of the full
  # image and label tensors, except the first dimension is now batch_size
  # rather than the full size of the train or test data sets.
  images_placeholder = tf.placeholder(tf.float32,
				      shape=(batch_size, gIMAGE_PIXELS))
  labels_placeholder = tf.placeholder(tf.int32, shape=(batch_size))
  return images_placeholder, labels_placeholder

In [165]:
def fill_feed_dict(data_set, images_pl, labels_pl):
  """Fills the feed_dict for training the given step.
  A feed_dict takes the form of:
  feed_dict = {
      <placeholder>: <tensor of values to be passed for placeholder>,
      ....
  }

  Args:
    data_set: The set of images and labels, from input_data.read_data_sets()
    images_pl: The images placeholder, from placeholder_inputs().
    labels_pl: The labels placeholder, from placeholder_inputs().

  Returns:
    feed_dict: The feed dictionary mapping from placeholders to values.
  """
  # Create the feed_dict for the placeholders filled with the next
  # `batch size` examples.
  images_feed, labels_feed = data_set.next_batch(gBatchSize, gFakeData)
  feed_dict = {
      images_pl: images_feed,
      labels_pl: labels_feed,
  }
  return feed_dict

In [166]:
def do_eval(sess,
            eval_correct,
            images_placeholder,
            labels_placeholder,
            data_set):
  """Runs one evaluation against the full epoch of data.

  Args:
    sess: The session in which the model has been trained.
    eval_correct: The Tensor that returns the number of correct predictions.
    images_placeholder: The images placeholder.
    labels_placeholder: The labels placeholder.
    data_set: The set of images and labels to evaluate, from
      input_data.read_data_sets().
  """
  # And run one epoch of eval.
  true_count = 0  # Counts the number of correct predictions.
  steps_per_epoch = data_set.num_examples // gBatchSize
  num_examples = steps_per_epoch * gBatchSize
  for step in xrange(steps_per_epoch):
    feed_dict = fill_feed_dict(data_set,
                               images_placeholder,
                               labels_placeholder)
    true_count += sess.run(eval_correct, feed_dict=feed_dict)
  precision = float(true_count) / num_examples
  print('  Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' %
        (num_examples, true_count, precision))

In [167]:
def run_training():
  """Train MNIST for a number of steps."""
  # Get the sets of images and labels for training, validation, and
  # test on MNIST.
  # Tell TensorFlow that the model will be built into the default Graph.
  with tf.Graph().as_default():
    # Generate placeholders for the images and labels.
    images_placeholder, labels_placeholder = placeholder_inputs(gBatchSize)

    # Build a Graph that computes predictions from the inference model.
    logits_op = inference(images_placeholder, gHidden1, gHidden2)

    # Add to the Graph the Ops for loss calculation.
    loss_op = loss(logits_op, labels_placeholder)

    # Add to the Graph the Ops that calculate and apply gradients.
    train_op = training(loss_op, gLearningRate)

    # Add the Op to compare the logits to the labels during evaluation.
    eval_correct_op = evaluation(logits_op, labels_placeholder)

    # Build the summary Tensor based on the TF collection of Summaries.
    summary = tf.summary.merge_all()

    # Add the variable initializer Op.
    init_op = tf.global_variables_initializer()

    # Create a saver for writing training checkpoints.
    saver = tf.train.Saver()

    # Create a session for running Ops on the Graph.
    sess = tf.Session()

    # Instantiate a SummaryWriter to output summaries and the Graph.
    summary_writer = tf.summary.FileWriter(gLogDir, sess.graph)

    # And then after everything is built:

    # Run the Op to initialize the variables.
    sess.run(init_op)

    # Start the training loop.
    for step in xrange(gMaxSteps):
      start_time = time.time()

      # Fill a feed dictionary with the actual set of images and labels
      # for this particular training step.
      feed_dict = fill_feed_dict(mnist_data.train,
                                 images_placeholder,
                                 labels_placeholder)

      # Run one step of the model.  The return values are the activations
      # from the `train_op` (which is discarded) and the `loss` Op.  To
      # inspect the values of your Ops or variables, you may include them
      # in the list passed to sess.run() and the value tensors will be
      # returned in the tuple from the call.
      _, loss_value = sess.run([train_op, loss_op],
                               feed_dict=feed_dict)

      duration = time.time() - start_time

      # Write the summaries and print an overview fairly often.
      if step % 100 == 0:
        # Print status to stdout.
        print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))
        # Update the events file.
        summary_str = sess.run(summary, feed_dict=feed_dict)
        summary_writer.add_summary(summary_str, step)
        summary_writer.flush()

      # Save a checkpoint and evaluate the model periodically.
      if (step + 1) % 1000 == 0 or (step + 1) == gMaxSteps:
        checkpoint_file = os.path.join(gLogDir, 'model.ckpt')
        saver.save(sess, checkpoint_file, global_step=step)
        # Evaluate against the training set.
        print('Training Data Eval:')
        do_eval(sess,
                eval_correct_op,
                images_placeholder,
                labels_placeholder,
                mnist_data.test)
        # Evaluate against the validation set.
        print('Validation Data Eval:')
        do_eval(sess,
                eval_correct_op,
                images_placeholder,
                labels_placeholder,
                mnist_data.validation)
        # Evaluate against the test set.
        print('Test Data Eval:')
        do_eval(sess,
                eval_correct_op,
                images_placeholder,
                labels_placeholder,
                mnist_data.test)

In [168]:
def set_up():
  if tf.gfile.Exists(gLogDir):
    tf.gfile.DeleteRecursively(gLogDir)
  tf.gfile.MakeDirs(gLogDir)
  run_training()

In [171]:
# tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)
set_up()

  Num examples: 10000  Num correct: 8966  Precision @ 1: 0.8966
Validation Data Eval:
  Num examples: 5000  Num correct: 4506  Precision @ 1: 0.9012
Test Data Eval:
  Num examples: 10000  Num correct: 8966  Precision @ 1: 0.8966


Step 1800: loss = 0.30 (0.001 sec)
Step 1900: loss = 0.29 (0.001 sec)
Training Data Eval:


Step 1500: loss = 0.44 (0.001 sec)
Step 1600: loss = 0.47 (0.001 sec)
Step 1700: loss = 0.39 (0.001 sec)


Step 1100: loss = 0.60 (0.024 sec)
Step 1200: loss = 0.37 (0.001 sec)
Step 1300: loss = 0.32 (0.001 sec)
Step 1400: loss = 0.41 (0.001 sec)


Training Data Eval:
  Num examples: 10000  Num correct: 8699  Precision @ 1: 0.8699
Validation Data Eval:
  Num examples: 5000  Num correct: 4340  Precision @ 1: 0.8680
Test Data Eval:
  Num examples: 10000  Num correct: 8699  Precision @ 1: 0.8699
Step 1000: loss = 0.43 (0.001 sec)


Step 700: loss = 0.62 (0.001 sec)
Step 800: loss = 0.56 (0.001 sec)
Step 900: loss = 0.58 (0.001 sec)


Step 400: loss = 1.27 (0.001 sec)
Step 500: loss = 0.82 (0.001 sec)
Step 600: loss = 0.72 (0.001 sec)


Step 0: loss = 2.30 (0.004 sec)
Step 100: loss = 2.09 (0.001 sec)
Step 200: loss = 1.83 (0.001 sec)
Step 300: loss = 1.50 (0.001 sec)


In [5]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the 'License');
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an 'AS IS' BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""A simple MNIST classifier which displays summaries in TensorBoard.
 This is an unimpressive MNIST model, but it is a good example of using
tf.name_scope to make a graph legible in the TensorBoard graph explorer, and of
naming summary tags so that they are grouped meaningfully in TensorBoard.
It demonstrates the functionality of every TensorBoard dashboard.
"""

'A simple MNIST classifier which displays summaries in TensorBoard.\n This is an unimpressive MNIST model, but it is a good example of using\ntf.name_scope to make a graph legible in the TensorBoard graph explorer, and of\nnaming summary tags so that they are grouped meaningfully in TensorBoard.\nIt demonstrates the functionality of every TensorBoard dashboard.\n'

In [7]:
mnist_data_dir = "/home/data/asarcar-data/tmp/MNIST_data"
mnist_data = input_data.read_data_sets(mnist_data_dir, one_hot=True, fake_data=gFakeData)

Extracting /home/data/asarcar-data/tmp/MNIST_data/train-images-idx3-ubyte.gz
Extracting /home/data/asarcar-data/tmp/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /home/data/asarcar-data/tmp/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /home/data/asarcar-data/tmp/MNIST_data/t10k-labels-idx1-ubyte.gz


In [7]:
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

In [8]:
def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
  """Reusable code for making a simple neural net layer.
  It does a matrix multiply, bias add, and then uses relu to nonlinearize.
  It also sets up name scoping so that the resultant graph is easy to read,
  and adds a number of summary ops.
  """
  # Adding a name scope ensures logical grouping of the layers in the graph.
  with tf.name_scope(layer_name):
    # This Variable will hold the state of the weights for the layer
    with tf.name_scope('weights'):
      weights = weight_variable([input_dim, output_dim])
      variable_summaries(weights)
    with tf.name_scope('biases'):
      biases = bias_variable([output_dim])
      variable_summaries(biases)
    with tf.name_scope('Wx_plus_b'):
      preactivate = tf.matmul(input_tensor, weights) + biases
      tf.summary.histogram('pre_activations', preactivate)
    activations = act(preactivate, name='activation')
    tf.summary.histogram('activations', activations)
    return activations

In [9]:
def train():
  sess = tf.InteractiveSession()

  # Define Layers
  # Input
  # Create a multilayer model.
  with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)
  
  hidden1 = nn_layer(x, 784, 500, 'layer1')

  with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('dropout_keep_probability', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

  # Do not apply softmax activation yet, see below.
  y = nn_layer(dropped, 500, 10, 'layer2', act=tf.identity)

  with tf.name_scope('cross_entropy'):
    # The raw formulation of cross-entropy,
    #
    # tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.softmax(y)),
    #                               reduction_indices=[1]))
    #
    # can be numerically unstable.
    #
    # So here we use tf.nn.softmax_cross_entropy_with_logits on the
    # raw outputs of the nn_layer above, and then average across
    # the batch.
    diff = tf.nn.softmax_cross_entropy_with_logits(y, y_)
    with tf.name_scope('total'):
      cross_entropy = tf.reduce_mean(diff)
  tf.summary.scalar('cross_entropy', cross_entropy)

  with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(gLearningRate).minimize(
        cross_entropy)

  with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
      accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  tf.summary.scalar('accuracy', accuracy)

  # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
  merged = tf.summary.merge_all()
  train_writer = tf.summary.FileWriter(gLogDir + '/train', sess.graph)
  test_writer = tf.summary.FileWriter(gLogDir + '/test')

  init_op = tf.global_variables_initializer()
  sess.run(init_op)

  def feed_dict(train):
    """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
    if train or gFakeData:
      xs, ys = mnist_data.train.next_batch(100, fake_data=gFakeData)
      k = gDropOut
    else:
      xs, ys = mnist_data.test.images, mnist_data.test.labels
      k = 1.0
    return {x: xs, y_: ys, keep_prob: k}

  # Train the model, and also write summaries.
  # Every 10th step, measure test-set accuracy, and write test summaries
  # All other steps, run train_step on training data, & add training summaries
  for i in range(gMaxSteps):
    if i % 10 == 0:  # Record summaries and test-set accuracy
      summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
      test_writer.add_summary(summary, i)
      print('Accuracy at step %s: %s' % (i, acc))
    else:  # Record train set summaries, and train
      if i % 100 == 99:  # Record execution stats
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()
        summary, _ = sess.run([merged, train_step],
                              feed_dict=feed_dict(True),
                              options=run_options,
                              run_metadata=run_metadata)
        train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
        train_writer.add_summary(summary, i)
        print('Adding run metadata for', i)
      else:  # Record a summary
        summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
        train_writer.add_summary(summary, i)
  train_writer.close()
  test_writer.close()


In [10]:
def set_up():
  if tf.gfile.Exists(gLogDir):
    tf.gfile.DeleteRecursively(gLogDir)
  tf.gfile.MakeDirs(gLogDir)
  train()

In [0]:
set_up()